# Explore population density in the bay area

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Modules and Packages



In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

# Supporting code

In [2]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    costs = {}
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        costs['total_cost'] = total_cost
        costs['minutes'] = round(total_cost / 60.0,1)
    
    return(costs)
    

In [3]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [4]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    return(pd.DataFrame(cursor.fetchall(), columns =['zip', 'population']))
        

In [5]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)


In [6]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [7]:
cursor = connection.cursor()

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

session = driver.session(database="neo4j")

# Find all zip codes and population, within 1, 3, 5, and 10 miles of the Downtown Berkeley station

To start, we'll look at all of the zip codes and population of those zip codes that we can service from the Downtown Berkley station. We'll use these panda's dataframes to compare to other stations to ensure that we're reaching net new cusotmers.

In [8]:
downtown_berkley_1_mile = my_station_get_zips('Downtown Berkeley', 1)
downtown_berkley_3_mile = my_station_get_zips('Downtown Berkeley', 3)
downtown_berkley_5_mile = my_station_get_zips('Downtown Berkeley', 5)
downtown_berkley_10_mile = my_station_get_zips('Downtown Berkeley', 10)

downtown_berkley_1_mile

,zip,population
0,94702,17092
1,94703,21937
2,94704,29190
3,94709,11740
4,94720,2971


# Get list of all other stations excluding 'Downtown Berkley'

In [9]:
def get_stations():
    connection.rollback()
    
    query = "select station from stations where station <> 'Downtown Berkley'"

    cursor.execute(query)

    stations = []
    temp_stations = cursor.fetchall()
    for station in temp_stations:
        stations.append(station[0])
    
    return stations

# Fin all zip codes and population within a distence of every station

At least 75% of the population must be in zip codes that are not within the same distence of the Downtown Berkeley station

In [10]:
def find_net_new_customers(distance):
    """finds all stations where 50% of the population within the distence would be net new"""
    stations = get_stations()
    
    berkeley_zips = my_station_get_zips('Downtown Berkeley', distance)
    berkeley_pop = berkeley_zips['population'].sum()
    
    new_stations = []
    
    for station in stations:
        station_zips = my_station_get_zips(station, distance)
        
        new_pop = station_zips['population'].sum()
        overlap_zips = pd.DataFrame(berkeley_zips.merge(station_zips, right_on='zip', left_on='zip',how='inner'))
        
        overlap_zip_pop = overlap_zips['population_x'].sum()
        
        percent_new = ( new_pop / overlap_zip_pop ) if overlap_zip_pop != 0 else 0
        
        if percent_new < .75:
            new_stations.append(station)
    
    return new_stations

In [11]:
stations_outside_distance = {}
for c in find_net_new_customers(8):
    stations_outside_distance[c] = my_neo4j_shortest_path('depart Downtown Berkeley','arrive ' + str(c))['minutes']

stations_outside_distance

{'Antioch': 61.0,
 'Berryessa': 70.0,
 'Concord': 35.0,
 'Dublin': 49.9,
 'Fremont': 50.0,
 'Millbrae': 60.0,
 'Milpitas': 65.0,
 'North Concord': 38.0,
 'Pittsburg': 44.0,
 'Pittsburg Center': 54.0,
 'SFO': 57.8,
 'San Bruno': 53.0,
 'South Hayward': 40.0,
 'Union City': 45.0,
 'Warm Springs': 56.0,
 'West Dublin': 46.9}

In [12]:
stations_outside_train_length = {}

shorest = stations_outside_distance[list(stations_outside_distance.keys())[0]]

for i in list(stations_outside_distance.keys()):
    if stations_outside_distance[i] < shorest:
        shorest = stations_outside_distance[i]

for c in get_stations():
    if my_neo4j_shortest_path('depart Downtown Berkeley','arrive ' + str(c))['minutes'] > shorest:
        stations_outside_train_length[c] = my_neo4j_shortest_path('depart Downtown Berkeley','arrive ' + str(c))['minutes']

In [13]:
list(stations_outside_train_length.keys())

['Antioch',
 'Balboa Park',
 'Berryessa',
 'Castro Valley',
 'Colma',
 'Daly City',
 'Dublin',
 'Fremont',
 'Glen Park',
 'Hayward',
 'Millbrae',
 'Milpitas',
 'North Concord',
 'Pittsburg',
 'Pittsburg Center',
 'SFO',
 'San Bruno',
 'South Hayward',
 'South San Francisco',
 'Union City',
 'Warm Springs',
 'West Dublin']

In [45]:
def pop_heatmap_by_station(stations, distance):
    """this funcation takes a list of stations and returns a dataframe for the heatmap"""
    heatmap_df = pd.DataFrame(columns=['latitude', 'longitude'])
    
    for s in stations:
        station_population = my_station_get_zips(s, distance)['population'].sum()
        
        connection.rollback()
    
        query = "select latitude, longitude from stations "
        query += "where station = '" + s + "'"

        cursor.execute(query)

        connection.rollback()

        rows = cursor.fetchall()

        for row in rows:
            latitude = row[0]
            longitude = row[1]
        
        new_pop = [[latitude, longitude]] * int(station_population)
        
        heatmap_df = heatmap_df.append(pd.DataFrame(new_pop, columns=['latitude', 'longitude']), ignore_index=True )
    
    return(heatmap_df)

In [46]:
heatmap_df = pop_heatmap_by_station(list(stations_outside_train_length.keys()), 5)
heatmap_df

9165652


,latitude,longitude
0,37.996281,-121.783404
1,37.996281,-121.783404
2,37.996281,-121.783404
3,37.996281,-121.783404
4,37.996281,-121.783404
...,...,...
9165647,37.699726,-121.928273
9165648,37.699726,-121.928273
9165649,37.699726,-121.928273
9165650,37.699726,-121.928273


In [ ]:
# look at customers who haven't made a sale yet
# find the zip codes where we have customers but they don't come into the store very often

In [61]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    count( distinct( c.customer_id ) ) as customers,
    sum(s.total_amount) as revenue
    
from customers c
    join sales s on c.customer_id = s.customer_id
    join stores st on c.closest_store_id = st.store_id
    
where st.city = 'Berkeley'

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,customers,revenue
0,8126,25041060


In [54]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with cte0 as (
    select
        s.customer_id,
        count(distinct(s.sale_id)) as purchases

    from customers c
        join sales s on c.customer_id = s.customer_id
    
    group by 1
)
select
    sum(purchases) / count(distinct(customer_id)) as avg_purchases
from cte0


"""

df_purchases = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
avg_orders_month = float(df_purchases.iloc[0] / 12)

print('The average customers makes ' + str(round(avg_orders_month,2)) + ' orders per month')

The average customers makes 4.13 orders per month


# Find number of potential customers by new location for mapping

Sum population within 7.5 miles of Daily City that are not within 7.5 miles of Berkeley
Sum popluation within 8 miles of the san jose station (the operating distence of our delivery droves)

In [59]:
locations = {'Berryessa':8,'Daly City':7.5,'Antioch':7.5, 'Dublin':7.5}
berkeley_zip = my_station_get_zips('Downtown Berkeley', 7.5)

total_customers = 0

for l in list(locations.keys()):
    new_customers = my_station_get_zips(l, locations[l])
    customers = new_customers['population'].sum() - new_customers.merge(berkeley_zip, right_on='zip', left_on='zip')['population_x'].sum()
    print(l + ': ' + str(customers))
    total_customers+=customers

print('Total: ' + str(total_customers))


Berryessa: 1181963
Daly City: 1108404
Antioch: 325681
Dublin: 374831
Total: 2990879


# Scenario Planning

Base: population * .02 * 12 * 5 / 30 * avg_orders_month == revenue

Bear: population * .0015 * 12 * 5 / 30 * avg_orders_month == revenue

Bull: population * .025 * 12 * 5 / 30 * avg_orders_month == revenue

In [63]:
base_estimation = float(total_customers) * .02 * 12 * 5 / 30 * avg_orders_month
bear_estimation = float(total_customers) * .015 * 12 * 5 / 30 * avg_orders_month
bull_estimation = float(total_customers) * .025 * 12 * 5 / 30 * avg_orders_month

print('Base: $' + str(round(base_estimation, 2)) )
print('Bear: $' + str(round(bear_estimation, 2)) )
print('Bull: $' + str(round(bull_estimation, 2)) )

Base: $493748.87
Bear: $370311.66
Bull: $617186.09
